# dedupe and finalize the list 

This notebook dedupes and finalizes the list

In [1]:
import intake 
import pandas as pd 
import numpy as np

In [2]:
cat = intake.open_catalog('catalog.yml')

In [3]:
# let's load the mills source datasets. 
mills_official = cat.mills_official.read()
# also the crosswalks

trillium_crosswalk = cat.trillium_mills_crosswalk.read()[['trillium_agency_name', 'mills_name']]
ctsa_crosswalk = cat.ctsa_mills_crosswalk.read()[['ctsa_agency_name', 'mills_name']]

In [4]:
# drop dupes b/c fiscal year means multiple rows in teh dataset
mills_official = mills_official.drop_duplicates(subset='Entity Name')

In [5]:
# assign "is_mills_act" to true for this subset
mills_official = mills_official.assign(is_mills_act=True)

In [6]:
mills_official = mills_official.rename({'Entity Name': 'mills_name'}, axis=1)

# Crosswalk / Join 

Using a manually created [crosswalk file](https://docs.google.com/spreadsheets/d/1inAKMhCkd4mtRcY1urM36IfLDzkyPRM1D3OJ_ftSXaw/edit?usp=sharing) we join the entire dataset to produce the final master list. 

In [7]:
# merge strategy 

# outer join CTSA / Mills, then left join trillium with mills. Then add in anything that is left from Trillium via concat.  

trillium_has_mills = trillium_crosswalk[trillium_crosswalk.mills_name.notnull()]
merged = (pd.concat([trillium_crosswalk[trillium_crosswalk.mills_name.isna()],
            (mills_official.merge(ctsa_crosswalk, 
                                  left_on='mills_name',
                                  right_on='mills_name',
                                  how='outer')
                           .merge(trillium_has_mills,
                                 left_on='mills_name',
                                 right_on='mills_name',
                                 how='left'))
                 ]).reset_index()
                   .drop('index', axis=1))
merged.head()

,trillium_agency_name,mills_name,is_mills_act,ctsa_agency_name
0,Maywood Dial-A-Ride,NaN,NaN,NaN
1,Whittier Dial-A-Ride,NaN,NaN,NaN
2,Inglewood,NaN,NaN,NaN
3,Monrovia Transit Dial-A-Ride,NaN,NaN,NaN
4,Azusa Transit,NaN,NaN,NaN


In [8]:
merged = merged.assign(is_mills_act = merged.is_mills_act.replace({np.nan: False}),
              mills_name = merged.mills_name.replace({np.nan: None}),
              ctsa_agency_name = merged.ctsa_agency_name.replace({np.nan: None}),
              trillium_agency_name = merged.trillium_agency_name.replace({np.nan: None})
             )

In [13]:
# quickly check no weirdness
assert(len(merged) == 211) #what I am expecting, may need to change as dataset changes

In [9]:
merged.to_csv('final/final_combined_paratransit_list.csv', index=False)